# Imports and Initialization of Config Class/File

In [ ]:
from io import open
import unicodedata
import string
import re
import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from distance import levenshtein
import os
import math
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils import data
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
#from torchtext.data import BucketIterator


seed = '4'
if seed is not None:
    random.seed(seed)

In [ ]:
#cfg class which is easliy translatible into a cfg file

class Config:
    seed = '5'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dict_name = 'rcrl_apd.1.4.1.txt'
    epochs = 200
    batch_size = 128
    hidden_dim = 128
    embed_dim = 64
    dropout = 0.5
    dec_max_len = 30
    MAX_LENGTH = 20
    teacher_forcing_ratio = 0.5
    n_layers = 2
    lr = 0.001


cfg = Config()

In [ ]:
print(cfg.device)

# Preparation of Datasets(using torchtext)

In [ ]:
def dict_sorting (dict_file_name):
    dict_file = open(dict_file_name, 'r')
    lines_dict = dict_file.readlines()
    dict_file.close()

    graphemes = []
    phonemes = []

    for i in range(0, len(lines_dict)):
        lines_dict[i] = lines_dict[i].split()
        graphemes.append([*lines_dict[i][0]])
        phonemes.append(lines_dict[i][1:])
    phonemes = [a for b in phonemes for a in b]
    graphemes = [a for b in graphemes for a in b]
    graphemes = sorted(set(graphemes))
    phonemes = sorted(set(phonemes))
    return graphemes, phonemes

dict_file_name = cfg.dict_name
g_seq, p_seq = dict_sorting(dict_file_name)
cfg.graphemes = ["<pad>", "<unk>", "</s>"] + g_seq
cfg.phonemes = ["<pad>", "<unk>", "<s>", "</s>"] + p_seq

#Index to grapheme and phones for vectors
cfg.graph2index = {g: idx for idx, g in enumerate(cfg.graphemes)}
cfg.index2graph = {idx: g for idx, g in enumerate(cfg.graphemes)}

cfg.phone2index = {p: idx for idx, p in enumerate(cfg.phonemes)}
cfg.index2phone = {idx: p for idx, p in enumerate(cfg.phonemes)}


print(cfg.graphemes)
print(cfg.phonemes)
cfg.g_vocab_size = len(cfg.graphemes)
cfg.p_vocab_size = len(cfg.phonemes)
print(cfg.g_vocab_size, cfg.p_vocab_size)
print(cfg.phone2index)
print(cfg.index2phone)

In [186]:
def DataLoading(dict_file_name, data_split):
    def sortingWP (d):
        w, p = [], []
        for i in range(0, len(d)):
            #w.append(d[i][0])
            w.append(' '.join(d[i][0]))
            p.append((' '.join(d[i][1:])))
        return w,p

    with open(dict_file_name) as f:
        dict_lines = f.readlines()
    vocab_len = len(dict_lines)
    print(vocab_len)
    random.shuffle(dict_lines)

    #Potential to add 'n to train dataset
    
    for i in range(0, len(dict_lines)):
        dict_lines[i] = dict_lines[i].split()
    train_data_lines, test_data_lines, eval_data_lines = [], [], []
    train_data_lines = dict_lines[0:int(data_split[0]*vocab_len)]
    test_data_lines = dict_lines[int(data_split[0]*vocab_len):int((data_split[0]+data_split[1])*vocab_len)]
    eval_data_lines = dict_lines[int((data_split[0]+data_split[1])*vocab_len):]

    train_word, train_phonemes = sortingWP(train_data_lines)
    test_word, test_phonemes = sortingWP(test_data_lines)
    eval_word, eval_phonemes = sortingWP(eval_data_lines)
    
    return train_word, train_phonemes, test_word, test_phonemes, eval_word, eval_phonemes

In [189]:
split = [0.8, 0.1, 0.1]
train_word, train_phonemes, test_word, test_phonemes, eval_word, eval_phonemes = DataLoading(cfg.dict_name, split)

#Sanity Check
print(len(test_phonemes))
print(len(train_phonemes))
print(len(eval_phonemes))
print(eval_word[0])
print(eval_phonemes[0])

24175
2417
19340
2418
a n t i r e t r o v i r a l e
a n t i r E t r u v i r A: l @


# Data Encoder & Decoder
converts data to their dictionary equivalents based on indices(And decoder which will be used when finally checking sequences)

In [ ]:
def data_encoder(seq, isWord):
    # Automatically encoders sequence with graph2index if words
    tokenized_seq = []
    if isWord: 
        seq = [*seq] + ['</s>']
        seq = [i for i in seq if i!=" "]
        for i in seq:
            a = cfg.graph2index[i]
            tokenized_seq.append(a)
    #Else simply add end of sequence token to to phoneme sequences
    else:
        a = '<s> ' + str(seq) +' </s>'
        seq = a.split(" ")
        ans = ""
        for i in seq:
            if i== 'o': i="O"
            elif i== 'h': i="h_"
            a = cfg.phone2index[i]
            #ans = ans +", " + a
            tokenized_seq.append(a)
        #tokenized_seq = ans

    #Tokenize sequence
    return tokenized_seq

In [ ]:
print(train_word[0])
print(train_phonemes[0])
a = data_encoder(train_word[0], 1)
b = data_encoder(train_phonemes[0], 0)
print(a)
print(b)

In [ ]:
def data_decoder(sequence, isWord):
    """Converts index sequence back into corresponding letter tokens"""
    if isWord: tokenizer = cfg.index2graph
    else: tokenizer = cfg.index2phone
    converted_sequence = []
    
    for i in sequence:
        if tokenizer[i] == "</s>": break
        a = tokenizer[i]
        converted_sequence.append(a)
    return converted_sequence

In [ ]:
a = [14, 4, 19, 23, 8, 12, 17, 2]
b = [27, 17, 31, 34, 8, 30, 3]
a = data_decoder(a, 1)
b = data_decoder(b, 0)
print(a)
print(b)

# Dataset Class 
(Adapted from https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class)

In [ ]:
class G2PData (data.Dataset):
    def __init__(self, graphemes, phonemes):
        self.graphemes = graphemes
        self.phonemes = phonemes

    def __len__(self):
        return len(self.graphemes)

    def __getitem__(self, index):
        graphemes = self.graphemes[index]
        phonemes = self.phonemes[index]

        #Fetches encoded versions
        grapheme_vector = data_encoder(graphemes, 1)
        phoneme_vector = data_encoder(phonemes, 0)

        #Omits </s> character
        decoder_inputs = phoneme_vector[:-1]
        phoneme_vector = phoneme_vector[1:]

        #Used for padding purposes
        g_vec_len = len(grapheme_vector) 
        p_vec_len = len(phoneme_vector)
        
        return grapheme_vector, phoneme_vector, decoder_inputs, g_vec_len, p_vec_len


In [ ]:
trainDataset = G2PData(train_word, train_phonemes)
testDataset = G2PData(test_word, test_phonemes)
evalDataset = G2PData(eval_word, eval_phonemes)

Padding: To ensure datasets in the same batch are of the same length (Could also use bucketiterator to choose strings)

# Model Setup
Based on seq2seq tutorial for Machine Translation (https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb)

Potential implementation of Attention: https://www.kaggle.com/code/omershect/learning-pytorch-seq2seq-with-m5-data-set/notebook

In [ ]:
class Encoder(nn.Module):
    def __init__(self, embed_dim, hidden_dim, g_vocab_size, n_layers, dropout):
        super().__init__()
        self.embed = embed_dim
        self.hidden = hidden_dim
        self.embed = nn.Embedding(g_vocab_size, embed_dim)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, batch_first = True)
    
    def forward(self, graph_seq, graph_seq_len):
        embed_inputs = self.embed(graph_seq)
        inputs = self.dropout(embed_inputs)

        #https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html
        #packs padded sequences into tensor
        input_tensor = pack_padded_sequence(inputs, graph_seq_len, batch_first=True, enforce_sorted=False)
        output, (hidden, context) = self.lstm(input_tensor)

        return hidden, context

In [ ]:
class Decoder(nn.Module):
    def __init__(self, embed_dim, hidden_dim, p_vocab_size, n_layers, dropout):
        super().__init__()
        self.embed = embed_dim
        self.hidden = hidden_dim
        self.embed = nn.Embedding(p_vocab_size, embed_dim)
        self.dropout = nn.Dropout(dropout) 
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim ,p_vocab_size) #Predicts output
    
    def forward(self, decoder_inputs, hidden_init, context_init):

        
        embed_inputs = self.embed(decoder_inputs)
        inputs = self.dropout(embed_inputs)

        #is already a tensor

        output, (hidden, context) = self.lstm(inputs, (hidden_init, context_init))


        #Scaling output
        activation_output = self.fc(output)
        
        
        return activation_output, hidden,context

In [ ]:
class G2PModel(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.enc = encoder
        self.dec = decoder
        self.device = device
        
    def forward(self, graph_seq, graph_seq_len,phone_seq_len, decoder_inputs, phoneme_target_vec = None, training = False, teacher_forcing = cfg.teacher_forcing_ratio):
        
        #Obtain hidden and context vectors from encoder
        hidden_init, context_init = self.enc(graph_seq, graph_seq_len)
        hidden, context = hidden_init, context_init

        max_len = max(phone_seq_len)

        phoneme_input_vec = decoder_inputs[:, :1]
        outputs = [] 
        phone_pred_seq = []
            
        if training:
            for i in range(0, max_len):

                output, hidden, context = self.dec(phoneme_input_vec ,hidden, context)
                outputs.append(output)
                # phone_pred = torch.tensor(output.argmax(-1))

                if random.random() > teacher_forcing: 
                    phoneme_input_vec = phoneme_target_vec[:,i]
                    
                else:  phoneme_input_vec = decoder_inputs[:,i]
                phoneme_input_vec = torch.unsqueeze(phoneme_input_vec,1)

        else: #for evaluation/prediction
            for i in range(1, cfg.dec_max_len+1):
                output, hidden, context = self.dec(phoneme_input_vec ,hidden, context)
                
                phone_pred = output.argmax(-1)
                outputs.append(output)
                phone_pred_seq.append(phone_pred)
                phoneme_input_vec = phone_pred
                #print(i)
                #print(phoneme_input_vec.shape)
            phone_pred_seq = torch.cat(phone_pred_seq, 1)
            


        output = torch.cat(outputs, 1)
        
        return output, phone_pred_seq
        
    

In [ ]:
# def init_weights(m):
#     for name, param in m.named_parameters():
#         nn.init.uniform_(param.data, -0.08, 0.08)
        
# model.apply(init_weights)

# Iterators Initialization and Padding

Padding_data takes a batch and pads it for every iteration

In [ ]:
def padding_data(batch):

    #Each sequence has a form:
    # grapheme_vector, phoneme_vector, decoder_inputs, g_vec_len, p_vec_len, graphemes, phonemes

    def get_components(batch, index):
        ans = []
        for i in batch:
            ans.append(i[index])
        return ans
    
    def pad_seq(batch, index, max_len):
        #ans = np.empty(cfg.batch_size)
        ans = []
        no_zeros_to_add = 0
        for i in batch:
            no_zeros_to_add = max_len - len(i[index])
            ans.append(i[index] + [0] * no_zeros_to_add)
        return torch.LongTensor(ans)
    
    grapheme_lens = [len(g[0]) for g in batch]

    phonemes_lens = [len(p[1]) for p in batch]

    input_maxlen = max(grapheme_lens)
    output_maxlen = max(phonemes_lens)
    padded_inputs = pad_seq(batch, 0, input_maxlen)
    padded_outputs = pad_seq(batch, 1, output_maxlen)
    padded_decoder_inputs = pad_seq(batch, 2, output_maxlen)

    return padded_inputs, padded_outputs, padded_decoder_inputs, grapheme_lens, phonemes_lens

In [ ]:
#Data Loader Implementation
#Shuffling not required as already loaded in a shuffled manner

train_iter =  data.DataLoader(trainDataset,batch_size=cfg.batch_size, shuffle=True, collate_fn=padding_data)
test_iter = data.DataLoader( testDataset,batch_size=cfg.batch_size, shuffle=False, collate_fn=padding_data)
eval_iter = data.DataLoader(evalDataset,batch_size=cfg.batch_size, shuffle=False, collate_fn=padding_data)

# Training and Evaluation Functions

Model, Optimizer and Criterion Initializer

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    model.train() #sets model in training model
    
    loss_epoch = 0
    

    for i, batch in enumerate(iterator):
        grapheme_vector, phoneme_vector, decoder_inputs, g_vec_len, p_vec_len = batch

        
        #Placing vectors in GPU to streamline process
        grapheme_vector = grapheme_vector.to(device)
        phoneme_vector = phoneme_vector.to(device)
        decoder_inputs = phoneme_vector.to(device)

        optimizer.zero_grad() #Sets all gradients to zero


        phoneme_pred,_ = model(grapheme_vector, g_vec_len,p_vec_len, decoder_inputs,phoneme_vector, True)

        #phoneme_pred is in shape (batchsize, max_len, p_vocab_size) -> need to drop the last diameter

        phoneme_pred = phoneme_pred.view(-1, phoneme_pred.shape[-1])
        phoneme_vector = phoneme_vector.view(-1)

        loss = criterion(phoneme_pred, phoneme_vector)

        

        loss.backward()

        optimizer.step()

        #Print loss every 50 batches
        # if (i % 50 == 0) and (i != 0) and i <100 : print(f" {i} batches completed: train loss: {loss}")
        # elif (i % 50 == 0) and (i != 0) : print(f"{i} batches completed: train loss: {loss}")


        #Normalizing Loss per batch
        loss_epoch += loss.item()/len(batch)


    
    return loss_epoch
        
        




In [ ]:
def WER_PER_calculation(phoneme_vector_text, phoneme_predict_text):
    "Word Error Rate - Words that are not 100% correct"
    "PER - Rate at which phonemes are incorrectly generated - Levenshtein distance/n_phones in correct text"
    n_wrong_words =   0
    total_phones_real = 0
    n_incorrect_phones = 0

    n_entries = len(phoneme_vector_text)

    for i in range(n_entries):
        total_phones_real += len(phoneme_vector_text[i])
        ans = levenshtein(phoneme_vector_text[i], phoneme_predict_text[i])
        n_incorrect_phones += ans
        if ans !=0 : n_wrong_words +=1
        
    #phoneme_error_rate = incorrect_phonestotal_phones_real - incorrect as doesnt allow a sensible way to work out PER over entire epoch
    #word_error_rate = n_wrong_words/n_entries
    
    return n_incorrect_phones, total_phones_real, n_wrong_words, n_entries

    

In [ ]:
def evaluate(model, iterator, criterion, device):

    def pad_p_vec(phoneme_vector, max_len):
        phoneme_vector = phoneme_vector.cpu().numpy()
        
        ans = []

        for i in phoneme_vector:
            no_zeros_to_add = (max_len - len(i))
            j = np.array(no_zeros_to_add * [0])
            j = j.reshape((no_zeros_to_add, 1))
            i = i.reshape((len(i), 1))

            ans.append(np.concatenate((i, j)))
            
        return torch.LongTensor(ans)


    model.eval()

    loss_epoch = 0
    n_incorrect_phones, total_phones_real, n_wrong_words, n_entries = 0, 0, 0, 0

    with torch.no_grad():

        for i, batch in enumerate(iterator):
            
            grapheme_vector, phoneme_vector, decoder_inputs, g_vec_len, p_vec_len = batch

            grapheme_vector = grapheme_vector.to(device)
            phoneme_vector = phoneme_vector.to(device)
            decoder_inputs = phoneme_vector.to(device)


            #graph_seq, graph_seq_len, decoder_inputs, training = False, teacher_forcing = cfg.teacher_forcing_ratio

            phoneme_pred, phoneme_pred_sequence = model(grapheme_vector, g_vec_len,p_vec_len, decoder_inputs,phoneme_vector, False) #False means only prediction and no training & teacher-forcing
            #print(phoneme_pred.shape)
            #print(f"before:{phoneme_vector.shape}")

            phoneme_vector = pad_p_vec(phoneme_vector, cfg.dec_max_len)
            phoneme_vector_copy = phoneme_vector

            phoneme_vector_copy = phoneme_vector_copy.to(device)
            phoneme_pred = phoneme_pred.view(-1, phoneme_pred.shape[-1])
            phoneme_vector_copy = phoneme_vector_copy.view(-1)

            loss = criterion(phoneme_pred, phoneme_vector_copy)
            #print(f"after:{phoneme_vector.shape}")



            # First move tensor to CPU then to numpy array for decoding source: https://stackoverflow.com/questions/49768306/pytorch-tensor-to-numpy-array

            # print(f"batch: {i}")
            # # for j in range(5):
            text_real, text_pred = [], []
            #print(f"real {phoneme_vector.shape}")
            #print(f"pred {phoneme_pred_sequence.shape}")
            phoneme_vector = torch.squeeze(phoneme_vector)
            for j in range(len(batch)):
                text_real.append( data_decoder(phoneme_vector[j].cpu().numpy().tolist(), 0))
                text_pred.append(data_decoder(phoneme_pred_sequence[j].cpu().numpy().tolist(), 0))
    
            a, b, c, d = WER_PER_calculation(text_pred, text_real)

            #n_incorrect_phones, total_phones_real, n_wrong_words, n_entries

            n_incorrect_phones += a
            total_phones_real += b
            n_wrong_words += c
            n_entries += d



    
            loss_epoch += loss.item()/len(batch)
  
    word_error_rate = n_wrong_words / n_entries
    phoneme_error_rate = n_incorrect_phones / total_phones_real

    return loss_epoch, word_error_rate, phoneme_error_rate, n_wrong_words

    

In [ ]:
encoder = Encoder(cfg.embed_dim, cfg.hidden_dim, cfg.g_vocab_size, cfg.n_layers, cfg.dropout)
decoder = Decoder(cfg.embed_dim, cfg.hidden_dim, cfg.g_vocab_size, cfg.n_layers, cfg.dropout)

model = G2PModel(encoder, decoder, cfg.device)
print(torch.cuda.is_available())
model.to(device=cfg.device)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')
optimizer = optim.Adam(model.parameters(), lr=cfg.lr)
criterion = nn.CrossEntropyLoss(ignore_index=0) #Ignores index corresponding to padding

In [ ]:
model_name = f"models/G2pLSTM-e{cfg.embed_dim}h{cfg.hidden_dim}n{cfg.n_layers}.pt"
# torch.save(model.state_dict(), model_name)

In [ ]:
combined_losses = np.zeros([1, 6])
print(combined_losses)
combined_losses= np.vstack([combined_losses,(1,2,3,4,5,6)])
print(combined_losses)

In [ ]:
prev_best_loss = math.inf
#combined_losses = np.empty([cfg.epochs, 6])

model_name = f"models/G2pLSTM-e{cfg.embed_dim}h{cfg.hidden_dim}n{cfg.n_layers}.pt"
print("Training")
matrix_exists = 0


for i in range(0, 60):
    print(f"Epoch {i}")

    train_loss = train(model, train_iter, optimizer, criterion, cfg.device)
    
    eval_loss, word_error_rate, phoneme_error_rate, n_wrong_words = evaluate(model, eval_iter, criterion, cfg.device)

    if matrix_exists == 0:
        combined_losses = np.array((i, train_loss, eval_loss, word_error_rate, phoneme_error_rate, n_wrong_words))
        matrix_exists = 1
    else:
        combined_losses= np.vstack([combined_losses,(i, train_loss, eval_loss, word_error_rate, phoneme_error_rate, n_wrong_words)])

    print(f"train loss: {round(train_loss, 3)}    eval loss: {round(eval_loss, 3)}    WER: {round(word_error_rate, 3)}   PER: {round(phoneme_error_rate, 3)}    Incorrect words: {round(n_wrong_words, 3)}")
    if prev_best_loss > eval_loss:
        prev_best_loss = eval_loss
        best_epoch = i
        if i > 10:
            torch.save(model.state_dict(), model_name)



In [ ]:
print(best_epoch)

In [ ]:
# import matplotlib.pyplot as plt
# # from importlib import reload
# # plt=reload(plt)
plt.title('G2P LSTM Losses',fontsize=18)
plt.xlabel('Epoch',fontsize=14)
plt.ylabel('Loss',fontsize=14)

plt.plot(combined_losses[:,0], combined_losses[:,1], label="Train Loss")
plt.plot(combined_losses[:,0], combined_losses[:,2], label="Eval Loss")
plt.locator_params(axis="x", integer=True, tight=True)
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.plot(combined_losses[:,0], combined_losses[:,3], label="wer")
plt.title('G2P LSTM WER',fontsize=18)
plt.xlabel('Epoch',fontsize=14)
plt.ylabel('WER',fontsize=14)
plt.locator_params(axis="both", integer=True, tight=True)
plt.show()

In [ ]:
plt.plot(combined_losses[:,0], combined_losses[:,4], label="PER")
plt.locator_params(axis="both", integer=True, tight=True)
plt.title('G2P PER',fontsize=18)
plt.xlabel('Epoch',fontsize=14)
plt.ylabel('PER',fontsize=14)
plt.show()

In [ ]:
plt.title('G2P LSTM incorrect words',fontsize=18)
plt.xlabel('Epoch',fontsize=14)
plt.ylabel('number of incorrect words',fontsize=14)
plt.plot(combined_losses[:,0], combined_losses[:,5], label="number of incorrect words")
plt.locator_params(axis="both", integer=True, tight=True)
plt.show()

In [ ]:
evaluate(model, eval_iter, criterion, cfg.device)

Created with assistance from tutorials and adapted from Sources:
1. https://github.com/Kyubyong/g2p - used for base notebook layout and initial introduction into structure of a G2P model - had no mention of teacherforcing
2. https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e#58f2 - assistance in understanding various concepts and functions in [1]
2. https://github.com/bentrevett/pytorch-seq2seq
3. https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

4. Further assistance on DataLoader and BucketIterator from: https://colab.research.google.com/github/gmihaila/ml_things/blob/master/notebooks/pytorch/pytorchtext_bucketiterator.ipynb#scrollTo=ChQWVc4IUUPb
